## 다변량분석_MLR
- 1주차 파이썬 실습

In [1]:
%pip install -r 'requirements.txt'

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not open requirements file: [Errno 2] No such file or directory: "'requirements.txt'"


In [2]:
#필요한 모듈 import
from typing import Union
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import mean_absolute_percentage_error as MAPE

In [3]:
#performance evaluation function
def perf_eval_reg(y_true: Union[np.array, list], y_pred: Union[np.array,list])->list:
    """
    input:
    y_true : ndarray of shape (n_samples,)
    y_pred : ndarray of shape (n_samples,)

    Return:
    [rmse,mae,mape]: scores of regression in a list
    """
    rmse = MSE(y_true, y_pred, squared=False)
    mae = MAE(y_true, y_pred)
    mape = MAPE(y_true, y_pred)
    
    return [rmse,mae,mape]

In [4]:
#display result
def display_result(result_dict: dict, dataset_names:list, metric_names: list):
    """
    Input:
    result_dict : 데이터 셋 별 Regression 평가 결과 저장한 dictionary
    ex) {'Toyota Corolla':[0.1,0.5,0.3],...}
    dataset_names : 데이터 셋 이름 List
    ex) ['Toyota Corolla','Boston Housing']
    metric_names : Regression 평가 지표 이름 List
    ex) ['RMSE','MAE','MAPE']
    """
    result_df = pd.DataFrame(result_dict).T
    result_df.index = dataset_names
    result_df.colums = metric_names

    display(result_df)

result_dict = {}


In [5]:
corolla = pd.read_csv("ToyotaCorolla.csv")

#indices for the activated input variables
nCar = len(corolla.index)#셈플 개수
nVar = len(corolla.columns)#총 변수의 개수

In [6]:
#1-to-c coding
dummy_p = np.array([0]*nCar)
dummy_d = np.array([0]*nCar)
dummy_c = np.array([0]*nCar)
p_idx = corolla['Fuel_Type']=='Petrol'
d_idx = corolla['Fuel_Type']=='Diesel'
c_idx = corolla['Fuel_Type']=='CNG'
dummy_p[p_idx] = 1
dummy_d[d_idx] = 1
dummy_c[c_idx] = 1

In [7]:
p_idx

0       False
1       False
2       False
3       False
4       False
        ...  
1431     True
1432     True
1433     True
1434     True
1435     True
Name: Fuel_Type, Length: 1436, dtype: bool

In [10]:
Fuel = pd.DataFrame({'Petrol' : dummy_p, 'Diesel' : dummy_d, 'CNG': dummy_c})

total_variable_idx = [i for i in range(nVar)]
removal_variable_idx = [0,1,7]

selected_variable_idx = list(set(total_variable_idx)- set(removal_variable_idx))
# Prepare the data for MLR
corolla_mlr_data = pd.concat((corolla.iloc[:,selected_variable_idx],Fuel),axis=1)
#iloc에서 :, 는 row는 모두 가져가고 컬럼은 선택된것만.

In [11]:
corolla_mlr_data

,Price,Age_08_04,Mfg_Month,Mfg_Year,KM,HP,Met_Color,Automatic,cc,Doors,...,Radio,Mistlamps,Sport_Model,Backseat_Divider,Metallic_Rim,Radio_cassette,Tow_Bar,Petrol,Diesel,CNG
0,13500,23,10,2002,46986,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
1,13750,23,10,2002,72937,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
2,13950,24,9,2002,41711,90,1,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
3,14950,26,7,2002,48000,90,0,0,2000,3,...,0,0,0,1,0,0,0,0,1,0
4,13750,30,3,2002,38500,90,0,0,2000,3,...,0,1,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1431,7500,69,12,1998,20544,86,1,0,1300,3,...,0,1,1,1,0,0,0,1,0,0
1432,10845,72,9,1998,19000,86,0,0,1300,3,...,0,0,1,1,0,0,0,1,0,0
1433,8500,71,10,1998,17016,86,0,0,1300,3,...,0,0,0,1,0,0,0,1,0,0
1434,7250,70,11,1998,16916,86,1,0,1300,3,...,0,0,0,1,0,0,0,1,0,0


In [13]:
from sklearn.model_selection import train_test_split
seed = 12345
test_size = 0.3
corolla_trn_data, corolla_valid_data = train_test_split(corolla_mlr_data, test_size=test_size, random_state=seed)

In [15]:
x_trn = corolla_trn_data.iloc[:, 1:].to_numpy() #데이터프레임을 넘파이 형태로.
y_trn = corolla_trn_data.iloc[:, 0].to_numpy()

from sklearn.linear_model import LinearRegression
mlr = LinearRegression()

mlr.fit(x_trn,y_trn)
#R-sq 점수 가져오기
mlr.score(x_trn,y_trn) 


0.9145065498996539

In [18]:
x_test = corolla_valid_data.iloc[:,1:]
y_test = corolla_valid_data.iloc[:,0]

y_pred = mlr.predict(x_test)
result_dict['toyota'] = perf_eval_reg(y_test,y_pred)
dataset_names = ['Toyota Corolla']
metric_names = ['RMSE','MAE','MAPE']
display_result(result_dict, dataset_names, metric_names)


C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_29088/2690721853.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  result_df.colums = metric_names


,0,1,2
Toyota Corolla,1219.170176,874.365151,0.086482


In [19]:
boston_housing = pd.read_csv('BostonHousing.csv')

nHome = len(boston_housing.index)
nVar = len(boston_housing.columns)

seed = 12345
test_size=0.3
boston_trn_data, boston_valid_data = train_test_split(boston_housing, test_size=test_size, random_state=seed)

In [20]:
x_trn = boston_trn_data.iloc[:,:-1].to_numpy()
y_trn = boston_trn_data.iloc[:,-1].to_numpy()

mlr = LinearRegression()

mlr.fit(x_trn,y_trn)

mlr.score(x_trn,y_trn)


0.7334524497257682

In [22]:
x_test = boston_valid_data.iloc[:,:-1]
y_test = boston_valid_data.iloc[:,-1]

y_pred = mlr.predict(x_test)

result_dict['boston'] = perf_eval_reg(y_test,y_pred)
dataset_names = ['Toyota Corolla','Boston Housing']
metric_names = ['RMSE','MAE','MAPE']
display_result(result_dict, dataset_names, metric_names)



C:\Users\Public\Documents\ESTsoft\CreatorTemp/ipykernel_29088/2690721853.py:14: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  result_df.colums = metric_names


,0,1,2
Toyota Corolla,1219.170176,874.365151,0.086482
Boston Housing,4.816714,3.394030,0.174921
